Uma das apis do spark na linguagem R se utiliza da interface do dplyr, biblioteca do tidyverse para construção de queries que torna o processo de analise mais similar a estrutura do SQL ja discutido em cursos anteriores, primeiramente façamos uma revisão do pacote dplyr

In [ ]:
require(dplyr) #Importando a biblioteca


Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




Para saber sobre a biblioteca podemos executar o seguinte comando e assim obter mais informações

In [ ]:
?dplyr

A linguagem SQL possui diversos comandos entre esses:

- select 
- where
- group by
- sort

Estes mesmos comandos são oferecidos na biblioteca dplyr, mantendo assim a seguinte equivalencia

- select
- filter
- group_by
- arrange

Entre outros comandos como mutate para adicionar novas colunas, ou summarize quando se quer aplicar operações sobre agrupamentos

A seguir mostramos alguns exemplos de como realizar diversas consultas sobre uma base de dados



A Base:

Já apresentada no curso de aprendizado de maquina, a base de dados iris é uma base com descrições sobre especies de plantas contendo 4 atributos descritores destes. No ambiente R este dataset ja vem carregado então observemos este

In [ ]:
head(iris)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa


Um comando conhecido do SQL é o SELECT, normalmente estruturado da forma SELECT atributos from tabela.

Na sintaxe dplyr, podemos escrever isto de forma funcional, que seria:

- select(Tabela,atributos)

Ou utilizando o operador %>%, este indica que o que vem a esquerda deve ser passado como argumento pro que vem a direita.

A motivação deste operador é criar um codigo mais parecido com SQL em sua leitura.

Por exemplo selecionemos apenas os atributos Sepal.Length,Sepal.Width e Species

Na forma funcional

In [ ]:
head(select(iris,Sepal.Length,Sepal.Width,Species))

,Sepal.Length,Sepal.Width,Species
,<dbl>,<dbl>,<fct>
1,5.1,3.5,setosa
2,4.9,3.0,setosa
3,4.7,3.2,setosa
4,4.6,3.1,setosa
5,5.0,3.6,setosa
6,5.4,3.9,setosa


Na forma SQL-Like

In [ ]:
iris %>%
 select(Sepal.Length,Sepal.Width,Species) %>%
 head()

,Sepal.Length,Sepal.Width,Species
,<dbl>,<dbl>,<fct>
1,5.1,3.5,setosa
2,4.9,3.0,setosa
3,4.7,3.2,setosa
4,4.6,3.1,setosa
5,5.0,3.6,setosa
6,5.4,3.9,setosa


Vendo então que ambos os metodos geram os mesmos resultados, se torna questão de preferencia como construir o codigo. Podemos deixar esta query bem mais complexa, por exemplo calculemos uma nova variavel, esta é a razão entre Sepal.Length e Sepal.Width. Chamemos esta de Sepal.Ratio

In [ ]:
iris %>%
 select(Sepal.Length,Sepal.Width,Species) %>%
 mutate(Sepal.Ratio=Sepal.Length/Sepal.Width) %>%
 head()

,Sepal.Length,Sepal.Width,Species,Sepal.Ratio
,<dbl>,<dbl>,<fct>,<dbl>
1,5.1,3.5,setosa,1.457143
2,4.9,3.0,setosa,1.633333
3,4.7,3.2,setosa,1.468750
4,4.6,3.1,setosa,1.483871
5,5.0,3.6,setosa,1.388889
6,5.4,3.9,setosa,1.384615


Assim criamos uma nova coluna, poderiamos tambem selecionar todas as colunas menos algumas com o select utilizando o - na coluna. Por exemplo calculado a razão, não queremos mais as colunas Sepal Length e Sepal Width

In [ ]:
iris %>%
 select(Sepal.Length,Sepal.Width,Species) %>%
 mutate(Sepal.Ratio=Sepal.Length/Sepal.Width) %>%
 select(-Sepal.Length,-Sepal.Width) %>%
 head()

,Species,Sepal.Ratio
,<fct>,<dbl>
1,setosa,1.457143
2,setosa,1.633333
3,setosa,1.468750
4,setosa,1.483871
5,setosa,1.388889
6,setosa,1.384615


O processo acima seria gerado a partir de um encadeamento de queries SQL, outros processos ja vistos nas aulas seriam o de agrupamentos, vemos como utilizar a função group_by e summarize para calcular a media da variavel Sepal.Ratio para cada especie

In [ ]:
iris %>%
 select(Sepal.Length,Sepal.Width,Species) %>%
 mutate(Sepal.Ratio=Sepal.Length/Sepal.Width) %>%
 select(-Sepal.Length,-Sepal.Width) %>%
 group_by(Species) %>%
 summarise(Sepal.Ratio=mean(Sepal.Ratio)) %>%
 head()

Species,Sepal.Ratio
<fct>,<dbl>
setosa,1.470188
versicolor,2.160402
virginica,2.230453


O mesmo calculo poderia ter sido feito sem grupo, e neste caso calculariamos a media considerando todas as especies

In [ ]:
iris %>%
 select(Sepal.Length,Sepal.Width,Species) %>%
 mutate(Sepal.Ratio=Sepal.Length/Sepal.Width) %>%
 select(-Sepal.Length,-Sepal.Width) %>%
 summarise(Sepal.Ratio=mean(Sepal.Ratio)) %>%
 head()

,Sepal.Ratio
,<dbl>
1,1.953681


Outro calculo simples é o de contagem, com a função count(), note que o select nem sempre é nescessario

In [ ]:
iris %>% group_by(Species) %>% count()

Species,n
<fct>,<int>
setosa,50
versicolor,50
virginica,50


Assim vemos que diversas das operações SQL vistas até agora podem ser executadas utilizando a sintaxe dplyr dentro do ecosistema R. Um motivo que pode estar interessado em se utilizar disto seria a facil integração de funções do ambiente R na construção de suas queries, por exemplo calculemos a correlação de pearson entre Sepal.Length e Petal Length para cada Especie

In [ ]:
iris %>% 
  select(Sepal.Length,Petal.Length,Species) %>%
  group_by(Species) %>%
  summarize(pearson=cor(Sepal.Length,Petal.Length))

Species,pearson
<fct>,<dbl>
setosa,0.2671758
versicolor,0.7540490
virginica,0.8642247


Assim vemos que enquanto estas variaveis são pouco relacionadas em setosa, a correlação é mais forte em versicolor e virginica.




Por fim um outro atributo importance esta associado ao where do SQL, metodo usado para filtrar amostras dado um condicional. Na biblioteca dplyr o comando utilizado é o filter. Por exemplo qual o maior valor de Sepal.Length para petal length menor que 2

In [ ]:
iris %>%
  select(Sepal.Length,Petal.Length) %>%
  filter(Petal.Length<2) %>%
  summarise(Maximo=max(Sepal.Length))

Maximo
<dbl>
5.8


Ou me utilizando do ecosistema R, poderia por exemplo verificar se a media entre as colunas Sepal.Lenth e Petal.Length é diferente entre todas as especies que não são a setosa

In [ ]:
iris %>% 
filter(Species!='setosa') %>%
 group_by(Species) %>%
  summarize( pvalor=t.test(Sepal.Length,Petal.Length)$p.value )

Species,pvalor
<fct>,<dbl>
versicolor,7.913472e-31
virginica,9.104528e-14


# Exercicio
Para o seguinte exercicio, calcule a razão entre sepal width e length, faça o mesmo para petal length e width. depois por especie faça um teste de medias utilizando o teste t-student para verificar se a media da razão da sepa é menor que a media de razão da petala (Assuma que todas as hipoteses estatisticas são validas.)

In [ ]:
iris %>% select(Petal.Length,Species) %>%
    pivot_wider(names_from = Species, values_from = Petal.Length) 

Warning message:
“Values from `Petal.Length` are not uniquely identified; output will contain list-cols.
* Use `values_fn = list` to suppress this warning.
* Use `values_fn = {summary_fun}` to summarise duplicates.
* Use the following dplyr code to identify duplicates.
  {data} %>%
    dplyr::group_by(Species) %>%
    dplyr::summarise(n = dplyr::n(), .groups = "drop") %>%
    dplyr::filter(n > 1L)”


setosa,versicolor,virginica
<list>,<list>,<list>
"1.4, 1.4, 1.3, 1.5, 1.4, 1.7, 1.4, 1.5, 1.4, 1.5, 1.5, 1.6, 1.4, 1.1, 1.2, 1.5, 1.3, 1.4, 1.7, 1.5, 1.7, 1.5, 1.0, 1.7, 1.9, 1.6, 1.6, 1.5, 1.4, 1.6, 1.6, 1.5, 1.5, 1.4, 1.5, 1.2, 1.3, 1.4, 1.3, 1.5, 1.3, 1.3, 1.3, 1.6, 1.9, 1.4, 1.6, 1.4, 1.5, 1.4","4.7, 4.5, 4.9, 4.0, 4.6, 4.5, 4.7, 3.3, 4.6, 3.9, 3.5, 4.2, 4.0, 4.7, 3.6, 4.4, 4.5, 4.1, 4.5, 3.9, 4.8, 4.0, 4.9, 4.7, 4.3, 4.4, 4.8, 5.0, 4.5, 3.5, 3.8, 3.7, 3.9, 5.1, 4.5, 4.5, 4.7, 4.4, 4.1, 4.0, 4.4, 4.6, 4.0, 3.3, 4.2, 4.2, 4.2, 4.3, 3.0, 4.1","6.0, 5.1, 5.9, 5.6, 5.8, 6.6, 4.5, 6.3, 5.8, 6.1, 5.1, 5.3, 5.5, 5.0, 5.1, 5.3, 5.5, 6.7, 6.9, 5.0, 5.7, 4.9, 6.7, 4.9, 5.7, 6.0, 4.8, 4.9, 5.6, 5.8, 6.1, 6.4, 5.6, 5.1, 5.6, 6.1, 5.6, 5.5, 4.8, 5.4, 5.6, 5.1, 5.1, 5.9, 5.7, 5.2, 5.0, 5.2, 5.4, 5.1"


In [ ]:
t.test(List$setosa,List$virginica)

Warning message in mean.default(x):
“argument is not numeric or logical: returning NA”


ERROR: ignored